# BART Hyperparameter Optimization for Topic Classification

Systematic hyperparameter optimization for BART-MNLI models on tweet-note classification.  
Tests 264 configurations to identify optimal training parameters.

**Approach**: Intelligent parameter sampling + systematic evaluation + performance analysis  
**Output**: Optimal configuration for production model training

## Dependencies and Configuration
Setting up imports, device configuration, and random seeds for reproducible experiments.

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from transformers import (
    BartForSequenceClassification, 
    BartTokenizer, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback,
    AdamW
)
from datasets import Dataset
import random
import gc
import json
import pickle
from datetime import datetime
import itertools
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Configuration
TRAIN_FILE = "Training set labeled.xlsx"
TEST_FILE = "Test set labeled.xlsx"
RANDOM_STATE = 42
MODEL_NAME = "facebook/bart-large-mnli"
EXCLUDED_CLASSES = ['other']  # Exclude 'other' class based on thesis findings

# Device setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Set seeds for reproducibility
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_STATE)

# Create output directories
os.makedirs('hyperparameter_results', exist_ok=True)

## Hyperparameter Search Space
Defining the comprehensive parameter grid for systematic exploration.  
Tests 264 combinations across learning rates, batch sizes, epochs, and architectural parameters.

In [ ]:
HYPERPARAMETER_GRID = {
    # Core hyperparameters (highest impact)
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5],
    'batch_size': [2, 4, 8, 16],
    'num_epochs': [3, 5, 7, 10],
    'weight_decay': [0.0, 0.01, 0.1],
    
    # Model architecture parameters
    'max_token_length': [512, 1024],
    'dropout_rate': [0.1, 0.2, 0.3],
    
    # Training parameters
    'warmup_steps': [0, 100, 500],
    'max_grad_norm': [0.5, 1.0, 2.0],
    'gradient_accumulation_steps': [1, 2, 4],
    
    # Regularization
    'label_smoothing': [0.0, 0.1, 0.2],
    'lr_scheduler_type': ['linear', 'cosine', 'polynomial'],
    
    # Technical parameters
    'fp16': [False, True],
    'optimizer_type': ['adamw'],  # Focus on AdamW for consistency
    'dataloader_num_workers': [0, 2],
}

## Utility Functions
Helper functions for experiment logging, intelligent parameter combination generation, and progress tracking.

In [ ]:
def log_experiment(message, log_file='hyperparameter_results/optimization_log.txt'):
    """Log experiment progress with timestamp"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"[{timestamp}] {message}"
    print(log_message)
    
    with open(log_file, 'a') as f:
        f.write(log_message + '\n')

def generate_intelligent_combinations(hyperparameter_grid: Dict, max_combinations: int = 264):
    """
    Generate intelligent hyperparameter combinations avoiding incompatible settings
    
    This function creates combinations in two phases:
    1. Priority combinations focusing on most impactful parameters
    2. Random exploration for broader coverage
    """
    
    log_experiment("Generating intelligent hyperparameter combinations...")
    
    # Phase 1: Priority configurations (most important hyperparameters)
    priority_params = {
        'learning_rate': hyperparameter_grid['learning_rate'],
        'batch_size': [4, 8],  # Focus on reasonable batch sizes
        'num_epochs': [5, 7],  # Focus on effective epoch counts
        'weight_decay': [0.0, 0.01],
        'max_token_length': [1024],  # Based on thesis results
        'lr_scheduler_type': ['linear', 'cosine'],
        'optimizer_type': ['adamw']
    }
    
    # Generate priority combinations
    priority_combinations = []
    priority_keys = list(priority_params.keys())
    priority_values = [priority_params[key] for key in priority_keys]
    
    for combination in itertools.product(*priority_values):
        config = dict(zip(priority_keys, combination))
        priority_combinations.append(config)
    
    log_experiment(f"Generated {len(priority_combinations)} priority combinations")
    
    # Phase 2: Complete configurations with intelligent defaults
    intelligent_combinations = []
    
    for base_config in priority_combinations[:max_combinations//2]:
        full_config = base_config.copy()
        
        # Intelligent parameter selection based on other parameters
        batch_size = full_config['batch_size']
        lr = full_config['learning_rate']
        
        # Memory-conscious settings
        if batch_size <= 4:
            full_config['gradient_accumulation_steps'] = random.choice([2, 4])
            full_config['fp16'] = True
        else:
            full_config['gradient_accumulation_steps'] = random.choice([1, 2])
            full_config['fp16'] = random.choice([True, False])
        
        # Learning rate dependent settings
        if lr >= 3e-5:
            full_config['warmup_steps'] = random.choice([100, 500])
            full_config['max_grad_norm'] = random.choice([0.5, 1.0])
        else:
            full_config['warmup_steps'] = random.choice([0, 100])
            full_config['max_grad_norm'] = random.choice([1.0, 2.0])
        
        # Add remaining parameters
        full_config.update({
            'dropout_rate': random.choice([0.1, 0.2]),
            'label_smoothing': random.choice([0.0, 0.1]),
            'dataloader_num_workers': 0,  # Safe default for compatibility
        })
        
        intelligent_combinations.append(full_config)
    
    # Phase 3: Random exploration
    random_combinations = []
    all_keys = list(hyperparameter_grid.keys())
    
    for _ in range(max_combinations - len(intelligent_combinations)):
        random_config = {}
        for key in all_keys:
            random_config[key] = random.choice(hyperparameter_grid[key])
        
        # Apply compatibility fixes
        if random_config['batch_size'] >= 16 and random_config['max_token_length'] == 1024:
            random_config['batch_size'] = 8  # Prevent OOM
        
        random_combinations.append(random_config)
    
    # Combine and deduplicate
    all_combinations = intelligent_combinations + random_combinations
    
    unique_combinations = []
    seen_configs = set()
    
    for config in all_combinations:
        config_str = json.dumps(config, sort_keys=True)
        if config_str not in seen_configs:
            seen_configs.add(config_str)
            unique_combinations.append(config)
    
    log_experiment(f"Generated {len(unique_combinations)} unique configurations")
    
    # Save combinations for reference
    with open('hyperparameter_results/all_combinations.json', 'w') as f:
        json.dump(unique_combinations, f, indent=2)
    
    return unique_combinations

## Data Loading and Preprocessing
Functions for loading, cleaning, and preparing balanced datasets.   
Handles text normalization, label encoding, and minority class oversampling.

In [ ]:
def load_and_prepare_data():
    """Load and prepare data for hyperparameter optimization"""
    
    def clean_dataframe(df):
        """Clean and preprocess dataframe"""
        df['label'] = df['label'].astype(str).str.lower().str.strip()
        df.dropna(subset=['label', 'tweet_text', 'note_text'], inplace=True)
        df = df[df['label'] != '']
        
        def clean_text(text):
            if pd.isna(text):
                return ""
            text = str(text).strip()
            # Remove non-printable characters
            text = ''.join(char for char in text if ord(char) >= 32 or char in '\t\n\r')
            return text
        
        df['tweet_text'] = df['tweet_text'].apply(clean_text)
        df['note_text'] = df['note_text'].apply(clean_text)
        df = df[(df['tweet_text'] != "") & (df['note_text'] != "")]
        
        return df.reset_index(drop=True)
    
    # Load and clean data
    df_train = clean_dataframe(pd.read_excel(TRAIN_FILE))
    df_test = clean_dataframe(pd.read_excel(TEST_FILE))
    
    # Filter out excluded classes from training
    df_train = df_train[~df_train['label'].isin(EXCLUDED_CLASSES)]
    
    # Oversample minority classes to balance dataset
    class_counts = df_train['label'].value_counts()
    max_count = class_counts.max()
    
    oversampled_dfs = []
    for label in class_counts.index:
        class_df = df_train[df_train['label'] == label]
        oversampled_df = class_df.sample(max_count, replace=True, random_state=RANDOM_STATE)
        oversampled_dfs.append(oversampled_df)
    
    df_train_balanced = pd.concat(oversampled_dfs).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
    
    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df_train_balanced['label'].unique())
    
    df_train_balanced['label_id'] = label_encoder.transform(df_train_balanced['label'])
    
    # Clean test data (exclude 'other' for evaluation)
    df_test_clean = df_test[~df_test['label'].isin(EXCLUDED_CLASSES)].copy()
    df_test_clean['label_id'] = label_encoder.transform(df_test_clean['label'])
    
    num_classes = len(label_encoder.classes_)
    
    log_experiment(f"Data prepared: {len(df_train_balanced)} train, {len(df_test_clean)} test samples")
    log_experiment(f"Classes: {list(label_encoder.classes_)}")
    
    return df_train_balanced, df_test_clean, label_encoder, num_classes

def create_datasets_with_config(df_train, df_test, config, model_name="facebook/bart-large-mnli"):
    """Create tokenized datasets with specific configuration"""
    
    tokenizer = BartTokenizer.from_pretrained(model_name)
    max_length = config['max_token_length']
    
    def tokenize_function(batch):
        # Using combined mode (tweet + note) based on thesis results
        return tokenizer(
            batch['tweet_text'], 
            batch['note_text'], 
            truncation="longest_first",
            padding="max_length", 
            max_length=max_length
        )

    # Create datasets
    train_dataset = Dataset.from_pandas(
        df_train[['tweet_text', 'note_text', 'label_id']].rename(columns={'label_id': 'labels'})
    )
    test_dataset = Dataset.from_pandas(
        df_test[['tweet_text', 'note_text', 'label_id']].rename(columns={'label_id': 'labels'})
    )
    
    # Tokenize
    train_dataset = train_dataset.map(tokenize_function, batched=True, batch_size=16)
    test_dataset = test_dataset.map(tokenize_function, batched=True, batch_size=16)
    
    # Remove text columns and prepare for training
    columns_to_remove = ['tweet_text', 'note_text']
    if '__index_level_0__' in train_dataset.column_names:
        columns_to_remove.append('__index_level_0__')
    
    train_dataset = train_dataset.remove_columns(columns_to_remove)
    test_dataset = test_dataset.remove_columns(columns_to_remove)
    
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    return train_dataset, test_dataset, tokenizer

## Model Training
Core training function that evaluates each hyperparameter configuration.   
Implements BART-MNLI training with early stopping and comprehensive error handling.

**Note**: ~39% success rate expected due to incompatible parameter combinations (typical in comprehensive search).

In [ ]:
def train_with_hyperparameters(train_dataset, test_dataset, config, num_classes, config_id):
    """Train model with specific hyperparameter configuration"""
    
    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Initialize model with dropout configuration
    model = BartForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_classes,
        ignore_mismatched_sizes=True,
        dropout=config['dropout_rate'],
        attention_dropout=config['dropout_rate'],
        activation_dropout=config['dropout_rate']
    )
    
    model.config.num_labels = num_classes
    model = model.to(DEVICE)
    
    # Calculate training parameters
    effective_batch_size = config['batch_size'] * config['gradient_accumulation_steps']
    total_steps = (len(train_dataset) // effective_batch_size) * config['num_epochs']
    warmup_steps = min(config['warmup_steps'], total_steps // 10)  # Cap warmup at 10%
    
    output_dir = f"hyperparameter_results/config_{config_id}"
    
    # Configure training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=config['num_epochs'],
        per_device_train_batch_size=config['batch_size'],
        per_device_eval_batch_size=config['batch_size'] * 2,
        gradient_accumulation_steps=config['gradient_accumulation_steps'],
        learning_rate=config['learning_rate'],
        weight_decay=config['weight_decay'],
        warmup_steps=warmup_steps,
        max_grad_norm=config['max_grad_norm'],
        seed=RANDOM_STATE,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        save_total_limit=1,
        report_to="none",
        logging_strategy="steps",
        logging_steps=100,
        dataloader_num_workers=config['dataloader_num_workers'],
        remove_unused_columns=False,
        fp16=config['fp16'],
        label_smoothing_factor=config['label_smoothing'],
        lr_scheduler_type=config['lr_scheduler_type'],
        dataloader_pin_memory=False,
        skip_memory_metrics=True,
        disable_tqdm=True
    )

    def compute_metrics(eval_pred):
        """Compute accuracy metric"""
        predictions, labels = eval_pred
        if isinstance(predictions, tuple):
            predictions = predictions[0]
        predictions = np.argmax(predictions, axis=1)
        accuracy = (predictions == labels).astype(np.float32).mean().item()
        return {"accuracy": accuracy}

    # Create optimizer
    optimizer = AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, None),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Train with error handling
    try:
        train_result = trainer.train()
        eval_result = trainer.evaluate()
        
        # Extract metrics
        final_train_loss = train_result.training_loss
        final_eval_accuracy = eval_result['eval_accuracy']
        final_eval_loss = eval_result['eval_loss']
        training_time = train_result.metrics.get('train_runtime', 0)
        
        log_experiment(f"✅ Config {config_id}: Accuracy={final_eval_accuracy:.4f}, Loss={final_eval_loss:.4f}")
        
        return {
            'config_id': config_id,
            'eval_accuracy': final_eval_accuracy,
            'eval_loss': final_eval_loss,
            'train_loss': final_train_loss,
            'training_time': training_time,
            'success': True,
            **config
        }
        
    except Exception as e:
        log_experiment(f"❌ Config {config_id} failed: {str(e)}")
        return {
            'config_id': config_id,
            'eval_accuracy': 0.0,
            'eval_loss': float('inf'),
            'train_loss': float('inf'),
            'training_time': 0,
            'success': False,
            'error': str(e),
            **config
        }
    
    finally:
        # Clean up memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

## Hyperparameter Optimization
Main optimization pipeline that systematically tests all 264 configurations.   
Tracks progress, handles failures, and identifies optimal parameters.

In [ ]:
def run_hyperparameter_optimization():
    """Run comprehensive hyperparameter optimization"""
    
    log_experiment("🚀 Starting hyperparameter optimization...")
    
    # Load data
    df_train, df_test, label_encoder, num_classes = load_and_prepare_data()
    
    # Generate hyperparameter combinations
    combinations = generate_intelligent_combinations(HYPERPARAMETER_GRID, max_combinations=264)
    
    log_experiment(f"🧪 Testing {len(combinations)} hyperparameter configurations...")
    
    all_results = []
    best_accuracy = 0.0
    best_config = None
    
    for i, config in enumerate(combinations):
        log_experiment(f"\n🔬 Testing configuration {i+1}/{len(combinations)}")
        log_experiment(f"Config: LR={config['learning_rate']}, BS={config['batch_size']}, "
                      f"Epochs={config['num_epochs']}, WD={config['weight_decay']}")
        
        try:
            # Create datasets
            train_dataset, test_dataset, tokenizer = create_datasets_with_config(
                df_train, df_test, config
            )
            
            # Train model
            result = train_with_hyperparameters(
                train_dataset, test_dataset, config, num_classes, i+1
            )
            
            all_results.append(result)
            
            # Track best configuration
            if result['success'] and result['eval_accuracy'] > best_accuracy:
                best_accuracy = result['eval_accuracy']
                best_config = result.copy()
                log_experiment(f"🏆 New best accuracy: {best_accuracy:.4f}")
            
            # Save checkpoint every 10 configurations
            if (i + 1) % 10 == 0:
                checkpoint_df = pd.DataFrame(all_results)
                checkpoint_df.to_csv(f'hyperparameter_results/results_checkpoint_{i+1}.csv', index=False)
                log_experiment(f"💾 Saved checkpoint at {i+1} configurations")
        
        except Exception as e:
            log_experiment(f"❌ Fatal error with config {i+1}: {str(e)}")
            continue
    
    # Save final results
    log_experiment("\n💾 Saving final results...")
    
    results_df = pd.DataFrame(all_results)
    results_df.to_csv('hyperparameter_results/hyperparameter_optimization_results.csv', index=False)
    results_df.to_excel('hyperparameter_results/hyperparameter_optimization_results.xlsx', index=False)
    
    # Save best configuration
    if best_config:
        with open('hyperparameter_results/best_configuration.json', 'w') as f:
            json.dump(best_config, f, indent=2)
        
        log_experiment(f"\n🏆 BEST CONFIGURATION FOUND:")
        log_experiment(f"Accuracy: {best_config['eval_accuracy']:.4f}")
        log_experiment(f"Learning Rate: {best_config['learning_rate']}")
        log_experiment(f"Batch Size: {best_config['batch_size']}")
        log_experiment(f"Epochs: {best_config['num_epochs']}")
        log_experiment(f"Weight Decay: {best_config['weight_decay']}")
    
    return results_df, best_config


## Results Analysis
Analysis and visualization functions for processing optimization results.  
Creates performance plots, parameter importance analysis, and top configuration rankings.

In [ ]:
def analyze_hyperparameter_results(results_df):
    """Analyze and visualize hyperparameter optimization results"""
    
    log_experiment("📊 Analyzing hyperparameter results...")
    
    # Filter successful runs
    successful_results = results_df[results_df['success'] == True].copy()
    
    if len(successful_results) == 0:
        log_experiment("❌ No successful runs to analyze")
        return
    
    log_experiment(f"📈 Analyzing {len(successful_results)} successful configurations")
    
    # Statistical summary
    accuracy_stats = successful_results['eval_accuracy'].describe()
    log_experiment(f"Accuracy Statistics:\n{accuracy_stats}")
    
    # Create analysis plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Learning Rate vs Accuracy
    axes[0, 0].scatter(successful_results['learning_rate'], successful_results['eval_accuracy'], alpha=0.7)
    axes[0, 0].set_xlabel('Learning Rate')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_title('Learning Rate vs Accuracy')
    axes[0, 0].set_xscale('log')
    
    # 2. Batch Size vs Accuracy
    batch_sizes = sorted(successful_results['batch_size'].unique())
    batch_accuracies = [successful_results[successful_results['batch_size'] == bs]['eval_accuracy'].values 
                       for bs in batch_sizes]
    axes[0, 1].boxplot(batch_accuracies, labels=batch_sizes)
    axes[0, 1].set_xlabel('Batch Size')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Batch Size vs Accuracy')
    
    # 3. Epochs vs Accuracy
    epochs = sorted(successful_results['num_epochs'].unique())
    epoch_accuracies = [successful_results[successful_results['num_epochs'] == ep]['eval_accuracy'].values 
                       for ep in epochs]
    axes[0, 2].boxplot(epoch_accuracies, labels=epochs)
    axes[0, 2].set_xlabel('Number of Epochs')
    axes[0, 2].set_ylabel('Accuracy')
    axes[0, 2].set_title('Epochs vs Accuracy')
    
    # 4. Weight Decay vs Accuracy
    weight_decays = sorted(successful_results['weight_decay'].unique())
    wd_accuracies = [successful_results[successful_results['weight_decay'] == wd]['eval_accuracy'].values 
                    for wd in weight_decays]
    axes[1, 0].boxplot(wd_accuracies, labels=weight_decays)
    axes[1, 0].set_xlabel('Weight Decay')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].set_title('Weight Decay vs Accuracy')
    
    # 5. Training Time vs Accuracy
    axes[1, 1].scatter(successful_results['training_time'], successful_results['eval_accuracy'], alpha=0.7)
    axes[1, 1].set_xlabel('Training Time (seconds)')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].set_title('Training Time vs Accuracy')
    
    # 6. Top 15 Configurations
    top_15 = successful_results.nlargest(15, 'eval_accuracy')
    axes[1, 2].barh(range(15), top_15['eval_accuracy'].values)
    axes[1, 2].set_yticks(range(15))
    axes[1, 2].set_yticklabels([f"Config {int(config_id)}" for config_id in top_15['config_id'].values])
    axes[1, 2].set_xlabel('Accuracy')
    axes[1, 2].set_title('Top 15 Configurations')
    
    plt.tight_layout()
    plt.savefig('hyperparameter_results/hyperparameter_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Parameter importance analysis
    log_experiment("\n📊 PARAMETER IMPORTANCE ANALYSIS:")
    log_experiment("="*50)
    
    for param in ['learning_rate', 'batch_size', 'num_epochs', 'weight_decay', 'dropout_rate']:
        if param in successful_results.columns:
            grouped = successful_results.groupby(param)['eval_accuracy'].agg(['mean', 'std', 'count'])
            best_value = grouped['mean'].idxmax()
            best_mean = grouped['mean'].max()
            worst_mean = grouped['mean'].min()
            impact = best_mean - worst_mean
            
            log_experiment(f"{param:15} | Impact: {impact:.4f} | Best: {best_value} ({best_mean:.4f})")
    
    # Save top configurations
    top_configs = successful_results.nlargest(20, 'eval_accuracy')
    top_configs.to_csv('hyperparameter_results/top_20_configurations.csv', index=False)
    
    return successful_results

def save_optimal_configuration(best_config, label_encoder):
    """Save optimal configuration without training final model"""
    
    log_experiment("💾 Saving optimal configuration for future use...")
    
    # Save configuration (match existing filename)
    with open('hyperparameter_results/best_configuration.json', 'w') as f:
        json.dump(best_config, f, indent=2)
    
    log_experiment(f"✅ Optimal configuration saved:")
    log_experiment(f"   Learning Rate: {best_config['learning_rate']}")
    log_experiment(f"   Batch Size: {best_config['batch_size']}")
    log_experiment(f"   Epochs: {best_config['num_epochs']}")
    log_experiment(f"   Weight Decay: {best_config['weight_decay']}")
    log_experiment(f"   Validation Accuracy: {best_config['eval_accuracy']:.4f}")
    
    return best_config

## Main Execution
Complete optimization pipeline execution: data preparation → configuration generation  
→ systematic evaluation → analysis → optimal configuration selection.

In [ ]:
def main():
    """Main execution function"""
    
    log_experiment("🚀 Starting comprehensive hyperparameter optimization for BART topic classification...")
    
    # Load data once
    df_train, df_test, label_encoder, num_classes = load_and_prepare_data()
    
    # Run hyperparameter optimization
    results_df, best_config = run_hyperparameter_optimization()
    
    # Analyze results if we have any
    if len(results_df) > 0:
        successful_results = analyze_hyperparameter_results(results_df)
        
        # Save optimal configuration for future use
        if best_config:
            save_optimal_configuration(best_config, label_encoder)
    
    # Print final summary
    log_experiment("🎉 Hyperparameter optimization completed!")
    
    print("\n" + "="*80)
    print("HYPERPARAMETER OPTIMIZATION SUMMARY")
    print("="*80)
    
    if best_config:
        print(f"🏆 BEST CONFIGURATION:")
        print(f"   Accuracy: {best_config['eval_accuracy']:.4f}")
        print(f"   Learning Rate: {best_config['learning_rate']}")
        print(f"   Batch Size: {best_config['batch_size']}")
        print(f"   Epochs: {best_config['num_epochs']}")
        print(f"   Weight Decay: {best_config['weight_decay']}")
        print(f"   Dropout Rate: {best_config['dropout_rate']}")
    
    successful_runs = len(results_df[results_df['success'] == True])
    print(f"\n📊 STATISTICS:")
    print(f"   Total configurations tested: {len(results_df)}")
    print(f"   Successful runs: {successful_runs}")
    print(f"   Success rate: {successful_runs/len(results_df)*100:.1f}%")
    
    print(f"\n📁 RESULTS SAVED TO:")
    print(f"   hyperparameter_results/hyperparameter_optimization_results.xlsx")
    print(f"   hyperparameter_results/best_configuration.json")
    print("="*80)

if __name__ == "__main__":
    main()

/home/hgb548/miniconda3/envs/bachelor_env_clean3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU: NVIDIA A100-PCIE-40GB
GPU Memory: 39 GB
[2025-05-27 23:47:25] 🚀 Starting comprehensive hyperparameter optimization...
[2025-05-27 23:47:25] 🧠 Generating intelligent hyperparameter combinations...
[2025-05-27 23:47:25] Generated 64 priority combinations
[2025-05-27 23:47:25] Generated 264 unique hyperparameter combinations
[2025-05-27 23:47:25] Target: 264 configurations


[2025-05-27 23:47:25] Data prepared: 924 train, 125 test samples
[2025-05-27 23:47:25] Classes: ['economics', 'health', 'lifestyle', 'politics', 'science', 'sports']
[2025-05-27 23:47:25] 🚀 Starting hyperparameter optimization...
[2025-05-27 23:47:25] Data prepared: 924 train, 125 test samples
[2025-05-27 23:47:25] Classes: ['economics', 'health', 'lifestyle', 'politics', 'science', 'sports']
[2025-05-27 23:47:25] 🧠 Generating intelligent hyperparameter combinations...
[2025-05-27 23:47:25] Generated 64 priority combinations
[2025-05-27 23:47:25] Generated 264 unique hyperparameter combinations
[2025-05-27 23:47:25] 🧪 Testing 264 hyperparameter configurations...
[2025-05-27 23:47:25] 
🔬 Testing configuration 1/264
[2025-05-27 23:47:25] Config: LR=1e-05, BS=4, Epochs=5, WD=0.0



Map:   0%|          | 0/924 [00:00<?, ? examples/s]


Map:   9%|▊         | 80/924 [00:00<00:01, 650.73 examples/s]

...

{'loss': 0.5888, 'grad_norm': 93.11883544921875, 'learning_rate': 1.9846849446849448e-05, 'epoch': 1.0822510822510822}


{'eval_loss': 1.2835172414779663, 'eval_accuracy': 0.7120000123977661, 'eval_runtime': 3.0682, 'eval_samples_per_second': 40.741, 'eval_steps_per_second': 5.215, 'epoch': 1.0822510822510822}


{'loss': 0.437, 'grad_norm': 1.4862374067306519, 'learning_rate': 1.937782587782588e-05, 'epoch': 1.2987012987012987}


{'eval_loss': 0.5898113250732422, 'eval_accuracy': 0.8320000171661377, 'eval_runtime': 2.9608, 'eval_samples_per_second': 42.218, 'eval_steps_per_second': 5.404, 'epoch': 1.2987012987012987}


{'loss': 0.2985, 'grad_norm': 0.29659485816955566, 'learning_rate': 1.88992303992304e-05, 'epoch': 1.5151515151515151}
